In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from dataset.dataset import DNADataset
from model.origami_model import DNAOrigamiModel
from model.transformer import EncoderLayer, TransformerEncoder
# from model.origami_model import DNAOrigamiModel
from utils.logger import get_logger
from utils.parsing import parse_dna_origami_data
from utils.tokenize import tokenize_trajectory
from oxDNA_analysis_tools.UTILS.RyeReader import describe, get_confs, inbox
import numpy as np

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
trajectory_filepaths = ["dataset/data/trajectory.dat"]
topology_filepaths = ["dataset/data/output.top"]

combined_data = tokenize_trajectory(parse_dna_origami_data(trajectory_filepaths[0], topology_filepaths[0]))

combined_data.shape

# dataset = DNADataset(trajectory_filepaths, topology_filepaths)
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

torch.Size([4, 86, 13])
torch.Size([2, 3655])
torch.Size([4, 86, 86])


IndexError: index 45 is out of bounds for dimension 0 with size 4

In [23]:
trajectory_filepaths = ["dataset/data/trajectory.dat"]
topology_filepaths = ["dataset/data/output.top"]

top_info, traj_info = describe(topology_filepaths[0], trajectory_filepaths[0])
n_confs = traj_info.nconfs
start_conf = 0
confs = get_confs(top_info, traj_info, start_conf, n_confs)

confs = [inbox(conf, center=True) for conf in confs]

pos = [conf.positions for conf in confs]
a1 = [conf.a1s for conf in confs]
a3 = [conf.a3s for conf in confs]

n_particles = pos[0].shape[0]
trajectory_data = np.array([pos, a1, a3]).reshape(n_confs,n_particles, -1)

In [3]:
data = dataset.graphs[0]['nodes']
data = data.reshape(1, data.shape[0], data.shape[1])

attn_mask = torch.ones(data.shape[0], data.shape[1])

In [4]:
embed = nn.Linear(13, 64)


In [5]:
embed(data).shape

torch.Size([1, 86, 64])

In [6]:
cfg = {'num_layers':1, 'n_features':13, 'd_model': 64, 'nhead': 1, 'num_encoder_layers': 6, 'd_ff': 256, 'dropout_rate': 0.1, 'device': 'cpu'}
encoder = TransformerEncoder(cfg)

In [7]:
encoder(data, attn_mask)

tensor([[[ 0.5675, -0.0164, -0.7847,  ...,  0.0181, -0.1691, -0.5160],
         [ 0.3918, -0.0264, -0.2521,  ..., -0.1603, -0.3439, -0.4047],
         [ 0.6419, -0.2373, -0.1428,  ...,  0.3467, -0.3449,  0.2875],
         ...,
         [ 1.0128,  1.4064,  1.0862,  ..., -0.5838, -1.1544,  1.9007],
         [ 0.6751,  1.3832,  1.1350,  ..., -0.5791, -1.1484,  1.9547],
         [-0.5929,  0.1272,  0.1006,  ..., -0.5408, -0.2063,  1.6927]]],
       grad_fn=<NativeLayerNormBackward0>)

In [8]:
# dataset.graphs[0]['nodes']

dataset.graphs[0]['edge_index'].shape

torch.Size([2, 3655])

In [9]:
encoder(img, return_embeddings = True)

NameError: name 'img' is not defined

In [ ]:
import torch
from x_transformers import ViTransformerWrapper, TransformerWrapper, Encoder, Decoder

encoder = ViTransformerWrapper(
    image_size = 86,
    patch_size = 43,
    attn_layers = Encoder(
        dim = 512,
        depth = 6,
        heads = 8
    )
)

# decoder = TransformerWrapper(
#     num_tokens = 20000,
#     max_seq_len = 1024,
#     attn_layers = Decoder(
#         dim = 512,
#         depth = 6,
#         heads = 8,
#         cross_attend = True
#     )
# )

img = torch.randn(1, 3, 256, 256)
img = dataset.graphs[0]['nodes'].reshape(1, 86, 13)
# caption = torch.randint(0, 20000, (1, 1024))

encoded = encoder(img, return_embeddings = True)
# decoder(caption, context = encoded) # (1, 1024, 20000)

EinopsError:  Error while processing rearrange-reduction pattern "b c (h p1) (w p2) -> b (h w) (p1 p2 c)".
 Input tensor shape: torch.Size([1, 86, 13]). Additional info: {'p1': 43, 'p2': 43}.
 Wrong shape: expected 4 dims. Received 3-dim tensor.

In [ ]:
dataset.graphs[0]['nodes'].shape

In [ ]:
pos = dataset.graphs[0]['nodes'][:, :3]

In [ ]:
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd

# Assuming pos is already defined
# pos = dataset.graphs[0]['nodes'][:, :3]

# Convert pos to a DataFrame for easier plotting
df = pd.DataFrame(pos, columns=['x', 'y', 'z'])

# Create a 3D scatter plot
fig = px.scatter_3d(df, x='x', y='y', z='z')

# Show the plot
fig.show()